<a href="https://colab.research.google.com/github/SChiu-project/reddit-NLP/blob/main/Data_extraction_praw_withVPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Submission ID
- full trading date between 1 JUl 2022 - 30 Jun 2023:  251
- submission id_extract: 253
- submission id_full: 256 - add back 3 missing submission_id: 5 Jun 2023, 15 Mar 2023, 25 Nov 2022
- 3 duplicate id: 6 Jun 2023, 6 Jun 2023, 16 Mar 2023
- 2 submission wthout trading date: 4 Jul 2022, 7 Apr 2023

Content
  - Pre 1: Load libraries
  - Pre 2: Connect VPN - optional
1. login to Reddit
2. search for submission IDs
3. Extract comments

In [1]:
pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 5.8 MB/s eta 0:00:00


In [2]:
pip install pandas

In [3]:
pip install numpy

In [4]:
# Pre 1: Load libraries
import praw
import pandas as pd
import numpy as np
import math
import datetime
import time

In [ ]:
# Pre 2: Connect VPN - optional
# manually log in to the NordVPN app on your computer before using the nordvpn-connect library

pip install nordvpn-connect

import nordvpn_connect

nordvpn_connect.nordvpn_connect()

nordvpn_connect.nordvpn_connect(server='us1234')

In [ ]:
# Pre 2: Connect VPN - optional
# disconnect from VPN

nordvpn_connect.nordvpn_disconnect()

## 1.login to Reddit

In [5]:
# login to Reddit
# Jun's account

reddit = praw.Reddit(
    client_id="btBlH-D9_rabq3hzoTuEpg",
    client_secret="lNFryDZ1Xf_7DKWWEgQ0ORdudkSP_A",
    user_agent="window:myredditproject:v1.1.1 (by u/Jun_Whelan)",
    check_for_async=False)

In [ ]:
# login to Reddit
# my account

reddit = praw.Reddit(
    client_id="UWyAG9YoinOXovOpixMH4A",
    client_secret="qN5D5IDQ-E3uVNJDpbgsyLFTZjug2w",
    user_agent="windows:PRAW_testing:v1.0 (by/u/GZespri)",
    check_for_async=False
)

In [6]:
# Make a test request to the Reddit API

start_time = time.time()
reddit.subreddit('test').hot(limit=1).__next__()

# Check the rate limit status
remaining_requests = reddit.auth.limits['remaining']
reset_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(reddit.auth.limits['reset_timestamp']))
requests_per_minute = reddit.auth.limits['remaining'] / ((reddit.auth.limits['reset_timestamp'] - time.time()) / 60)

print(f"Remaining requests: {remaining_requests}")
print(f"Reset time: {reset_time}")
print(f"Requests per minute: {requests_per_minute:.2f}")


Remaining requests: 599.0
Reset time: 2023-10-28 04:40:00
Requests per minute: 66.19


## 2. search for submission IDs

In [7]:
# search for all ' What Are Your Moves Tomorrow' threads
# limited result
subreddit_name = 'wallstreetbets'
keyword = 'What Are Your Moves Tomorrow'
submissions = reddit.subreddit(subreddit_name).search("title:{}".format(keyword), sort='new', limit = 1000) #no after

data = []
for submission in submissions:
    if keyword.lower() in submission.title.lower():
        data.append([submission.title, submission.id])

df = pd.DataFrame(data, columns=['title', 'submission_id'])
print(df)

                                               title submission_id
0     What Are Your Moves Tomorrow, October 27, 2023       17h56kj
1     What Are Your Moves Tomorrow, October 26, 2023       17gdls6
2     What Are Your Moves Tomorrow, October 25, 2023       17flt81
3     What Are Your Moves Tomorrow, October 24, 2023       17etjph
4     What Are Your Moves Tomorrow, October 23, 2023       17e1u9u
..                                               ...           ...
243  What Are Your Moves Tomorrow, November 04, 2022        yld24m
244  What Are Your Moves Tomorrow, November 03, 2022        ykfgej
245  What Are Your Moves Tomorrow, November 02, 2022        yjjigk
246  What Are Your Moves Tomorrow, November 01, 2022        yin4pn
247   What Are Your Moves Tomorrow, October 31, 2022        yhoa3q

[248 rows x 2 columns]


In [8]:
# save the list of submission id

df.to_csv("submissionID_period.csv")

In [9]:
# read the latest full list of submission id
df =pd.read_csv("submissionID_1Jul22-28Oct23_extract.csv",sep=",")
df.drop(["Unnamed: 0"],axis=1,inplace=True)
df

,title,submission_id
0,"What Are Your Moves Tomorrow, October 27, 2023",17h56kj
1,"What Are Your Moves Tomorrow, October 26, 2023",17gdls6
2,"What Are Your Moves Tomorrow, October 25, 2023",17flt81
3,"What Are Your Moves Tomorrow, October 24, 2023",17etjph
4,"What Are Your Moves Tomorrow, October 23, 2023",17e1u9u
...,...,...
331,"What Are Your Moves Tomorrow, July 18, 2022",w1fzcu
332,"What Are Your Moves Tomorrow, July 06, 2022",vs6pb2
333,"What Are Your Moves Tomorrow, July 01, 2022",voheqa
334,"What Are Your Moves Tomorrow, July 05, 2022",vrfxqk


## 3. Extract comments

In [10]:
# 定义一个函数，用于将 comment 对象转换为字符串
def process_comment(comment):
    return comment.body

# 定义一个函数，用于获取所有帖子的评论并将它们合并为一个字符串
def get_comments(submission_id):
    submission = reddit.submission(id=submission_id)
    submission.comments.replace_more(limit=None)
    comments = submission.comments.list()
    comment_texts = [process_comment(comment) for comment in comments]
    return ' '.join(comment_texts)

In [ ]:
# workaround to the API limit
# limit the comment extarction to 10 threads
df_temp = df[191:201]

# 对于数据框中的每个 submission_id，获取所有评论并将它们合并为一个字符串
comments = []
for submission_id in df_temp['submission_id']:
    comment_text = get_comments(submission_id)
    comments.append(comment_text)

# 将字符串添加到数据框中的新列
df_temp['comments'] = comments


df_temp.to_csv("df_temp.csv")

TooManyRequests: received 429 HTTP response